# 1. Create a Pipeline
## 1.1. Import dependencies

In [ ]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

## 1.2. Initialize components

In [ ]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
text_embedder = SentenceTransformersTextEmbedder()
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

## 1.3. Create the pipeline

In [ ]:
query_pipeline = Pipeline()

## 1.4. Add components

In [ ]:

# Here is an example of how you'd add the components initialized in step 2 above:
query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)

rag_pipeline.add_component("llm", llm)

## 1.5. Connect components

## 1.6. Run the pipeline
